In [1]:
import math
import datetime

import click
import numpy as np
import torch
from dgl.data.utils import save_graphs
from environment import Environment
from parameters import Parameters
from src.classes.dataset import Dataset
from src.get_embeddings import get_embeddings
from src.max_margin_loss import max_margin_loss
from src.model.conv_model import ConvModel
from src.train_loop import train_loop

from src.utils_data import assign_graph_features
from src.utils import read_data, save_txt, save_outputs
from src.utils_vizualization import plot_train_loss
from src.metrics import (create_already_bought, create_ground_truth,
                         get_metrics_at_k, get_recommandation_tensor)
from src.evaluation import explore_recs, explore_sports, check_coverage
from presplit import presplit_data

from logging_config import get_logger

log = get_logger(__name__)


In [2]:
environment = Environment()
parameters = Parameters({
    'aggregator_hetero': 'mean',
    'aggregator_type': 'mean',
    'clicks_sample': 0.3,
    'delta': 0.266,
    'dropout': 0.01,
    'hidden_dim': 256,
    'out_dim': 128,
    'embedding_layer': True,
    'batch_size': 10,
    'lr': 0.00017985194246308484,
    'n_layers': 5,
    'neg_sample_size': 2000,
    'norm': True,
    'use_popularity': True,
    'weight_popularity': 0.5,
    'days_popularity': 7,
    'purchases_sample': 0.5,
    'prediction_layer': 'cos',
    'use_recency': True,
    'num_workers': 1
})


In [3]:
# Create full train set
dataset = Dataset(
    environment, parameters
)

In [4]:
%load_ext autoreload
%autoreload 2
from src.create_graph import create_graph

graph = create_graph(dataset)

Version 3


In [5]:
dim_dict = {'customer': graph.nodes['customer'].data['features'].shape[1],
            'article': graph.nodes['article'].data['features'].shape[1],
            'edge': graph.edges['buys'].data['features'].shape[1],
            'out': parameters.out_dim,
            'hidden': parameters.hidden_dim}
dim_dict

{'customer': 20, 'article': 600, 'edge': 5, 'out': 128, 'hidden': 256}

In [6]:
model = ConvModel(graph,
                    dim_dict,
                    parameters
                    )


In [7]:
graph.device

device(type='cpu')

In [30]:
%reload_ext autoreload
%autoreload 2
from src.classes.dataloaders import DataLoaders

dataloaders = DataLoaders(graph,
                    dataset,
                    parameters,
                    environment
                    )

oh oho ho hu


In [9]:
if environment.cuda:
    print("Assigning model to GPU.")
    model = model.to(environment.device)

Assigning model to GPU.


In [10]:
    model.train_loss_list = []
    model.train_precision_list = []
    model.train_recall_list = []
    model.train_coverage_list = []
    model.val_loss_list = []
    model.val_precision_list = []
    model.val_recall_list = []
    model.val_coverage_list = []
    best_metrics = {}  # For visualization
    max_metric = -0.1
    patience_counter = 0  # For early stopping
    min_loss = 1.1

    opt = parameters.optimizer(model.parameters(),
                               lr=parameters.lr)


In [ ]:
model.train()

In [31]:
_, pos_g, neg_g, blocks = next(iter(dataloaders.dataloader_train_metrics))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f565d5bd8b0>
Traceback (most recent call last):
  File "/home/treb/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/treb/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/home/treb/anaconda3/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


RuntimeError: DataLoader worker (pid(s) 18197) exited unexpectedly